# Object References, Mutability, and Recycling

## Object References and Mutability

In Python, variables are essentially references to objects in memory. When you assign a value to a variable, you're creating a reference to the object. Understanding this reference behavior is essential to grasp how object mutability works.

## Variables Are Not Boxes

Consider variables as labels pointing to objects, rather than boxes storing values. This perspective is vital when dealing with mutable objects like lists and dictionaries.



With reference variables, it makes much more sense to say that the variable is assigned to an object, and not the other way around. After all, the object is created before the assignment.  
To understand an assignment in Python, read the righthand side first: that’s where the object is created or retrieved. After that, the variable on the left is bound to the object, like a label stuck to it. Just forget about the boxes.

In [1]:
a = [1, 2, 3]
b = a  # Bind the variable b to the same value that a is referencing
a.append(4)
b

[1, 2, 3, 4]

In [2]:
# Example 6-2. Variables are bound to objects only after the objects are created
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 140472343798896
Gizmo id: 140472342539104


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

In [3]:
z = x
z is x

True

## Identity, Equality, and Aliases

Every object in Python has an identity (unique identifier), which you can access using the id() function. You can check if two variables reference the same object using the is operator, or if their values are equivalent using the == operator. Sometimes, two variables can be aliases, meaning they reference the same object.


An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The is operator compares the identity of two objects; the id() function returns an integer representing its identity.  

The real meaning of an object’s ID is implementation dependent. In CPython, id() returns the memory address of the object, but it may be something else in another Python interpreter. The key point is that the ID is guaranteed to be a unique integer label, and it will never change during the life of the object.  

the most frequent use for id() is while debugging, when the repr() of two objects look alike, but you need to understand whether two references are aliases or point to separate objects.  

The `==` operator compares the values of objects (the data they hold), while is compares their identities.  

The is operator is faster than ‍‍`==` , because it cannot be overloaded, so Python does not have to find and invoke special methods to evaluate it, and computing is as simple as comparing two integer IDs. 

In [4]:
# Example 6-3. charles and lewis refer to the same object
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [5]:
id(charles), id(lewis)


(140288030783744, 140288030783744)

In [6]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

### Choosing Between == and is

Each object has its own `identity`, `type` and `value`. Only the identity does not change once it is created. The is operator compares the identity of two objects. (Use the id() function)

`is` is faster than `==` because is only compares the ids of two Objects with singletons like `None`. `a == b` is equivalent to the abbreviation of `a.__eq__(b)`. The` __eq__` method is inherited from objects that compare ids, but most build-in data types override the `__eq__` method to match their equality behavior.


Let's make it clear:

* `==`     compares values of objects
* `is`     compares identity of Objects


In [4]:
x is None

False

In [5]:
x is not None

True

## The Relative Immutability of Tuples

When we use mutable data types in a tuple, the mutable data type can change, but the reference to the tuple remains the same while the value of the list within it changes.

In [8]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [9]:
id(t1[-1])

140288030783232

In [10]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [11]:
id(t1[-1])

140288030783232

In [12]:
t1 == t2

False

## Copies Are Shallow by Default

When you create a copy of a mutable object using the assignment operator or a copy method like copy(), you get a shallow copy. This means that the top-level object is duplicated, but the inner objects are still references. For deep copies, you can use the copy module's deepcopy() function.


In [6]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1) # or l2 = l1[:]


In [7]:
l2

[3, [55, 44], (7, 8, 9)]

In [8]:
l2 == l1

True

In [9]:
l2 is l1

False

In [17]:
#example 6-6
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)

l1.append(100)
l1[1].remove(55)

print(f'l1 : {l1}')
print(f'l2 : {l2}\n')

l2[1] += [33, 22] # Changes the list in place so this change happens in l1 too.
l2[2] += (10, 11) # Creates a new tuple with new id so this change only happen in l2.

print(f'l1 : {l1}')
print(f'l2 : {l2}\n')

l1 : [3, [66, 44], (7, 8, 9), 100]
l2 : [3, [66, 44], (7, 8, 9)]

l1 : [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2 : [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]



### Deep and Shallow Copies of Arbitrary Objects

Note that making deep copies is not a simple matter in the general case. Objects may have cyclic references that would cause a naive algorithm to enter an infinite loop. The deepcopy function remembers the objects already copied to handle cyclic references gracefully.

In [11]:
# Example 6-8. Bus picks up and drops off passengers
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [12]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1) #  bus2 is a shallow copy of bus1
bus3 = copy.deepcopy(bus1)
bus1.drop('Bill')


In [13]:
bus2.passengers

['Alice', 'Claire', 'David']

In [14]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

#### Cyclic references

In [15]:
# Example 6-10. Cyclic references: b refers to a, and then is appended to a; deepcopy still manages to copy a
a = [10, 20]
b = [a, 30]
a.append(b)

In [16]:
a

[10, 20, [[...], 30]]

In [17]:
from copy import deepcopy
c = deepcopy(a)

In [18]:
c

[10, 20, [[...], 30]]

To customize the behavior of `copy()` or `deepcopy()` we must implement `__copy__()` or `__deepcopy__()`

## Function Parameters as References

Understanding how function parameters work as references is crucial for writing correct and efficient code.


Python call functions with call by sharing method.

Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become aliases of the actual arguments. 
The result of this scheme is that a function may change any mutable object passed as a parameter, but it cannot change the identity of those objects

In [26]:
# Example 6-11. A function may change any mutable object it receives
def f(a, b):
    a += b
    return a
x = 1
y = 2


In [27]:
f(x, y)

3

In [28]:
x, y

(1, 2)

In [29]:
a = [1, 2]
b = [3, 4]

In [30]:
f(a, b)

[1, 2, 3, 4]

In [31]:
a, b

([1, 2, 3, 4], [3, 4])

In [32]:
t = (10, 20)
u = (30, 40)

In [33]:
f(t, u)

(10, 20, 30, 40)

In [34]:
t, u

((10, 20), (30, 40))

## Mutable Types as Parameter Defaults: Bad Idea

When defining a function with mutable default arguments, such as lists or dictionaries, you should be cautious. The default value is evaluated only once, when the function is defined. This can lead to unexpected behavior, especially when multiple calls to the function share the same default object.


In [19]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [36]:
bus1 = HauntedBus(['Alice', 'Bill'])

In [37]:
bus1.passengers

['Alice', 'Bill']

In [38]:
bus1.pick('Charlie')
bus1.drop('Alice')

In [39]:
bus1.passengers

['Bill', 'Charlie']

In [40]:
bus2 = HauntedBus()
bus2.pick('Carrie')

In [41]:
bus2.passengers

['Carrie']

In [42]:
bus3 = HauntedBus()

In [43]:
bus3.passengers

['Carrie']

In [44]:
bus3.pick('Dave')

In [45]:
bus2.passengers

['Carrie', 'Dave']

In [46]:
bus2.passengers is bus3.passengers

True

In [47]:
bus1.passengers

['Bill', 'Charlie']

In [48]:
dir(HauntedBus.__init__)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [49]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [50]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

In [20]:
HauntedBus.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'A bus model haunted by ghost passengers',
              '__init__': <function __main__.HauntedBus.__init__(self, passengers=[])>,
              'pick': <function __main__.HauntedBus.pick(self, name)>,
              'drop': <function __main__.HauntedBus.drop(self, name)>,
              '__dict__': <attribute '__dict__' of 'HauntedBus' objects>,
              '__weakref__': <attribute '__weakref__' of 'HauntedBus' objects>})

## Defensive Programming with Mutable Parameters

When a function receives a mutable parameter, you should consider whether the caller expects the argument to be changed or not. This is about aligning the expectations of the coder of the function and the caller.


In [51]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [52]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

In [53]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [54]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

## del and Garbage Collection

The del statement in Python doesn't delete objects. Instead, it deletes references to objects. When the number of references to an object reaches zero, the object may be garbage collected. The __del__ special method, when defined, allows an object to release external resources before being destroyed.


In [21]:
a = [1, 2]
b = a
del a
b

[1, 2]

In [22]:
b = [3] #Rebinding b to a different object removes the last remaining reference to [1, 2].

**Note**
* There is a `__del__` special method, but it does not cause the disposal
of the instance, and **should not be called by your code**.
* `__del__` is invoked by the Python interpreter when the instance is
about to be destroyed to give it a chance to release external
resources. 
* You will seldom need to implement `__del__` in your
own code.
* The proper use of `__del__` is rather tricky. See the
`__del__` special method documentation in the “Data Model” chapter
of The Python Language Reference

In [23]:
# Example 6-16. Watching the end of an object when no more references point to it
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('...like tears in the rain.')

ender = weakref.finalize(s1, bye)
ender.alive

True

**Weak reference**

* `finalize` method holds a `weak reference` to `{1, 2, 3}`.
* Weak reference to an object does not increase its reference count.
* It does not prevent the target object from being garbage collected.
* Weak references are useful in caching applications. 

In [24]:
del s1
ender.alive

True

In [25]:
#Rebinding the last reference, s2, makes {1, 2, 3} unreachable.It is destroyed,the bye callback is invoked.
s2 = 'spam' 

...like tears in the rain.


In [26]:
ender.alive

False

In [61]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return f'Cheese({self.kind!r})'

In [62]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
           Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese


In [63]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [64]:
del catalog
sorted(stock.keys())

['Parmesan']

In [65]:
del cheese
sorted(stock.keys())

[]

## Tricks Python Plays with Immutables

Python has some unique behaviors when it comes to immutables like tuples, strings, bytes, and frozensets. For instance, creating a new tuple from another doesn't create a copy but returns a reference to the same object. String literals may also create shared objects as an optimization technique called interning.


In [27]:
t1 = (1, 2, 3)
t2 = tuple(t1)

In [28]:
t2 is t1

True

In [29]:
t3 = t1[:]
t3 is t1

True

In [30]:
# Example 6-18. String literals may create shared objects
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [31]:
# Surprise: a and b refer to the same str!
s1 = 'ABC'
s2 = 'ABC'
s2 is s1 

True


Remember to use == to compare strings or integers for equality instead of is. Interning is an optimization for internal use of the Python interpreter and should not impact your code.

Understanding object references, mutability, and recycling is crucial for writing clean and efficient Python code. These concepts enable you to work with Python's rich data structures effectively while avoiding common pitfalls and unexpected behavior.


# Lecturers

1. Mohammad Rajabi ..., [Linkedin](https://www.linkedin.com/in/mohammad-rajabi-me/)
2. Pooya Fekri, [Linkedin](https://www.linkedin.com/in/pooyafekri)


present date : 2023-10-27

# Reviewers

1. Hosein Toodehroosta, review date: 2023-10-26, [LinkedIn](https://www.linkedin.com/in/hossein-toodehroosta/)
2. Zohreh Alizadeh, review date: 2023-10-27, [LinkedIn](https://ir.linkedin.com/in/zohreh-bayramalizadeh/)
3. Mehran Faraji, review date: 2023-10-27, [LinkedIn](https://www.linkedin.com/in/mehranfaraji/)
4. Mahya Asgarian, review date: 2023-10-27, [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249/)
5. Saeed Hemmati, review date: 2023-10-27, [LinkedIn](https://www.linkedin.com/in/saeed-hemati/)